In [0]:
%pip install python-dotenv

In [0]:
from pyspark.sql.functions import *

from src.data.data_utils import write_delta_table
from src.data.geometry_utils import make_envelope, make_bbox
from src.data.image_utils import enrich_output
from src.data.token_utils import get_token

In [0]:
catalog_dev = "`land_auto-gen-kart_dev`"
schema_dev = "dl_bildesegmentering"
spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")
bronze_table = "endepunkt_bronze"
silver_table = "endepunkt_silver"

SUBDIR = {"image": "storreendepunkt_images", "dom": "storreendepunkt_dom"}

bbox_length = 128
image_width, image_height = 512, 512

In [0]:
q = f"""
CREATE TABLE IF NOT EXISTS {silver_table} (
    nodeid STRING,
    geometry BINARY,
    bbox ARRAY<DOUBLE>,
    bbox_str STRING,
    image_path STRING,
    dom_path STRING,
    image_wms STRING,
    dom_wms STRING,
    image_status STRING,
    dom_status STRING,
    kommune_id STRING,
    ingest_time TIMESTAMP,
    photo_time DATE
) USING DELTA
"""
spark.sql(q)

In [0]:
def read_table_to_wkt():
    """
    Leser polygonene fra bronsetabellen og returnerer dem som en GeoDataFrame med WKT geometri.
    """
    df = spark.read.table(bronze_table).withColumn(
        "geometry", expr("ST_GeomFromWKB(geometry)")
    )
    return df

In [0]:
token = get_token()

df = read_table_to_wkt()

df = make_envelope(df, bbox_length)
df = make_bbox(df, bbox_length)
df = df.withColumn("bbox_str", concat_ws(",", col("bbox")))

df = enrich_output(df, token, "nodeid", SUBDIR, image_width, image_height)
df = df.withColumn("geometry", expr("ST_AsBinary(geometry)"))

write_delta_table(df, silver_table, id_col="nodeid")